In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

## Loading the files

The City of Richmond Assessor's Office has a public data set of property ownership files.

You can read more about them and access XLSX files at their site: https://www.rva.gov/assessor-real-estate/data-request

Text/CSV files have historically been available at the FTP site: ftp://ftp.ci.richmond.va.us/Assessor/Real%20Tables/

In [72]:
def load_files():
    # property transfers
    tdf = pd.read_csv('data/COR Public Data April 14 2021/RealLand.txt', low_memory=False)
    tdf['PIN'] = tdf['PIN'].str.strip()
    
    # we also need the Master file, to get the property classes (LandType in tdf is mostly useless)
    mdf = pd.read_csv('data/COR Public Data April 14 2021/RealMaster.txt', low_memory=False)
    mdf['PIN'] = mdf['PIN'].str.strip()
    
    return (tdf, mdf)

In [106]:
def sanitize_addresses(mdf):
    # Sanitize location address to match mailing address
    mdf['LocAddr'] = mdf['LocAddr'].str.upper().str.strip()
    mdf['MailAddr'] = mdf['MailAddr'].str.strip()

    mdf['MailAddr'] = mdf['MailAddr'].str.replace('RD$','ROAD',regex=True)
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('STREET','ST')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace(' LN',' LANE')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('DRIVE','DR')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('TERR','TER')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('AVENUE','AVE')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('P.O.','PO',regex=False)
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('OXFORD CIRCLE WEST','W OXFORD CIR')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('OXFORD CIRCLE EAST','E OXFORD CIR')
    mdf['MailAddr'] = mdf['MailAddr'].str.replace('CARY ST ROAD','CARY STREET ROAD')
    
    # lazy method - see if first 4 characters of the addresses match. this usually catches the whole street number.
    mdf['loc_short'] = mdf['LocAddr'].str[0:4]
    mdf['mail_short'] = mdf['MailAddr'].str[0:4]
    
    return mdf

# looking up counts of different street names
#mdf[mdf['LocAddr'].str.upper().str.contains(' DR')].shape[0]
# total 75292

# MailAddr
# ROAD 1515
# RD 13899
# STREET 1863
# ST 19846
# LANE 1631
# LN 1429

# LocAddr
# ROAD 13887
# RD 0
# STREET 315
# ST 26628
# LANE 2593
# LN 0
# DR 4775
# DRIVE 0

In [78]:
(tdf, mdf) = load_files()
tdf.shape

(75292, 107)

In [107]:
# sanitized master
smdf = sanitize_addresses(mdf)

In [70]:
# reference - property classes. run this to create the file
#mdf.groupby(['PropClas','PCDesc'])['PIN'].count().reset_index().to_csv('property_class_ref.csv',index=False)

# Generally, 101-399 are R (residential) properties
# 101 = vacant
# 102 = island
# 105 = park
# 110-181 = most homes (55396 rows)
# 210-212 = condos (3908 rows)
# 310-341 = apartments
# 196,396 = LIHTC apartments

In [114]:
# Create a new column to identify whether it's a primary resident (ie, the owner's mailing addr matches the location)

# this one didn't work great - there's a lot of variation, e.g. "N0051081001	3201 FIFTH AVE	3201 5TH AVE"
#smdf['resOwner'] = 0
#smdf.loc[smdf.MailAddr == smdf.LocAddr, 'resOwner'] = 1
#smdf.loc[(smdf.PropClas==196) | smdf.PropClas.between(310, 355) | (smdf.PropClas == 396), 'resOwner'] = 0 # set rentals to 0

# this one is also not perfect - there are rare exceptions like:
# W0000103015	2 W FRANKLIN ST	2 W MAIN ST
# but it has a much higher accuracy than comparing full addresses - adds about 5k rows
smdf['resOwner'] = 0
smdf.loc[smdf.mail_short == smdf.loc_short, 'resOwner'] = 1
smdf.loc[(smdf.PropClas==196) | smdf.PropClas.between(310, 355) | (smdf.PropClas == 396), 'resOwner'] = 0 # set rentals to 0

# print how many owners (see tdf.shape for total addresses) currently 41396
smdf.loc[smdf.resOwner == 1].shape

(41396, 136)

In [134]:
# add a new column to indicate how far away the owner of the property lives
# 0 = owner-occupied
# 1 = same zip
# 2 = in richmond
# 3 = in virginia
# 4 = outside VA
smdf['ownerDistance'] = 'Owner out of state'
smdf.loc[smdf['MailStat'].str.strip() == 'VA','ownerDistance'] = 'Owner in Virginia'
smdf.loc[smdf['MailCity'].str.strip() == 'RICHMOND', 'ownerDistance'] = 'Owner in Richmond'
#smdf.loc[smdf['MailZip'].str.strip() == smdf['LocZip'].str.strip(), 'ownerDistance'] = 'Owner in same zip code'
smdf.loc[smdf['resOwner'] == 1, 'ownerDistance'] = 'Owner-occupied'

In [189]:
# merge into a combined df
# keep only homes (residential parcels)
homes = smdf.loc[smdf.PropClas.between(110, 181) | smdf.PropClas.between(196, 198) | smdf.PropClas.between(210, 212) \
                  | smdf.PropClas.between(310, 355) | (smdf.PropClas == 396) \
                 , ['PIN','PropClas','resOwner','ownerDistance','MailAddr','Owner1']]

# join dfs, only keep homes
cdf = pd.merge(tdf, homes, left_on=['PIN'], right_on=['PIN'], how='inner')

In [190]:
# columns to keep:
cols = ['LRSN','PIN','Sale1D','Sale2D','Sale3D','PropClas','resOwner','ownerDistance']
cdf = cdf.loc[:,cols]
cdf['Sale1D'] = pd.to_datetime(cdf['Sale1D'], format='%m/%d/%Y', errors='coerce') 
cdf['Sale2D'] = pd.to_datetime(cdf['Sale2D'], format='%m/%d/%Y', errors='coerce') 
cdf['Sale3D'] = pd.to_datetime(cdf['Sale3D'], format='%m/%d/%Y', errors='coerce') 

In [191]:
from datetime import date 

cdf['curr_own_len'] = pd.Timestamp.now().normalize() - cdf['Sale1D']

In [192]:
cdf = cdf.set_index(['LRSN','PIN'])
cdf['len2D'] = cdf['Sale1D']-cdf['Sale2D']
cdf['len3D'] = cdf['Sale2D']-cdf['Sale3D']
cdf['avg_own_len'] = cdf.loc[:,['curr_own_len','len2D','len3D']].mean(axis=1)
cdf = cdf.reset_index()
cdf.head()

,LRSN,PIN,Sale1D,Sale2D,Sale3D,PropClas,resOwner,ownerDistance,curr_own_len,len2D,len3D,avg_own_len
0,76223,C0070553183,2017-08-09,NaT,NaT,212,0,4,1528 days,NaT,NaT,1528 days 00:00:00
1,20318,E0000635006,2007-11-16,2002-12-02,2000-05-01,120,1,0,5082 days,1810 days,945 days,2612 days 08:00:00
2,74710,W0001283044,2018-03-07,2012-06-19,2005-10-18,211,0,3,1318 days,2087 days,2436 days,1947 days 00:00:00
3,1880,C0011224032,2009-04-03,2003-09-11,2002-01-22,110,1,0,4578 days,2031 days,597 days,2402 days 00:00:00
4,10053,C0070166130,2010-06-22,2010-03-08,2006-08-30,120,1,0,4133 days,106 days,1286 days,1841 days 16:00:00


In [127]:
# edie 64563
# NaTs - 76223
# glitchy? 29281
cdf.loc[cdf['LRSN']==64563].set_index('LRSN').unstack()

               LRSN 
PIN            64563            W0190082028
Sale1D         64563    2018-04-24 00:00:00
Sale2D         64563    2014-03-10 00:00:00
Sale3D         64563    2009-08-07 00:00:00
PropClas       64563                    150
resOwner       64563                      1
ownerDistance  64563                      0
curr_own_len   64563     1270 days 00:00:00
len2D          64563     1506 days 00:00:00
len3D          64563     1676 days 00:00:00
avg_own_len    64563     1484 days 00:00:00
dtype: object

In [193]:
cdf['curr_own_len'] = cdf['curr_own_len'].dt.days
cdf['avg_own_len'] = cdf['avg_own_len'].dt.days


In [141]:
# save to CSV
cdf.loc[:,['LRSN','PIN','curr_own_len','avg_own_len','PropClas','resOwner','ownerDistance']].to_csv('all_res_own.csv',index=False)

In [33]:
mdf.loc[mdf['PIN']=='E0000911003', ['PIN','MailAddr', 'LocAddr', 'resOwner', 'PropClas'] ]

,PIN,MailAddr,LocAddr,resOwner,PropClas
3719,E0000911003,900 E BROAD ST RM 701,2100 WHITCOMB ST,0,105


## for houses, do homeowners own them for longer than investors, or nah

In [66]:
tdf.loc[tdf.PropClas.between(110, 181) | mdf.PropClas.between(210, 212)].groupby('resOwner').mean()

,LRSN,PropClas,curr_own_len,avg_own_len
resOwner,,,,
0,37954.796755,123.090094,4085.537376,3880.925921
1,36290.031192,118.297107,5064.603500,4445.052762


In [67]:
tdf.loc[tdf.PropClas.between(110, 181) | mdf.PropClas.between(210, 212)].groupby('resOwner').median()

,LRSN,PropClas,curr_own_len,avg_own_len
resOwner,,,,
0,37112.5,115.0,2878.0,2948.0
1,36427.5,115.0,3421.0,3369.0


In [68]:
# no, definitely not - either way, secondary properties are typically sold 4-600 days sooner than primary

2948/365 # median 8 years for investors to sell, vs 9.2 for homeowners

8.076712328767123

## for the Distance metric, how many fall into each category?

In [170]:
smdf.loc[smdf['resOwner']==0].groupby(['ownerDistance']).count()['PIN'] / smdf.loc[smdf['resOwner']==0].count()['PIN']

ownerDistance
1    0.132405
2    0.447132
3    0.312721
4    0.107741
Name: PIN, dtype: float64

In [194]:
# looking at the top mailing addresses for residences
homes.groupby('MailAddr').agg({'Owner1':lambda x: set(x),'PIN':'count'}).sort_values('PIN',ascending=False).head(20)

,Owner1,PIN
MailAddr,,
5310 MARKEL RD #104,"{8 CANTERBURY TRUST TRUSTEE , CAVA CA...",148
2200 PENNSYLVANIA AVE NW 4 FL E,{AWE BROOKSIDE OWNER LLC },147
304 E MAIN ST,"{CLAIBORNE COURT LLC , DOBRIN ...",134
300 W FREEMASON ST,{VILLAS OF OAKWOOD LLC },129
901 CHAMBERLAYNE PKWY,{RICHMOND REDEVELOPMENT AND },113
4100 REGENT ST STE G,{RB HRIP RICHMOND MULTIFAMILY },108
23 W BROAD ST STE 100,"{4101 NORTH AVENUE II LLC , BECKSTO...",94
PO BOX 25989,"{TARTAKOVSKY STEVEN V , RVA REN...",94
PO BOX 69,"{SOLFERINO RAYMOND , SOLFERI...",93


In [166]:
for c in cc.glasbey_hv[0:5]:
    for i in c:
        print(i*255)

47.999925000000005
161.99997000000002
218.00001
251.999925
79.00002
47.999925000000005
228.999945
174.000015
56.00004
109.000005
144.00003
79.00002
138.99999
138.99999
138.99999


In [195]:
import colorama